# Tarea 2: análisis de sentimientos usando aprendizaje supervisado

##### Por: Daniela Flores Villanueva 

## Sobre las librerías

En primer lugar, se cargan todas las librerías empleadas en esta tarea:
- `pandas`: librería utilizada para cargar la matriz GloVe y posteriormente, para confeccionar un `DataFrame` con palabras y su sentimiento asociado.
- `csv`: utilizada para cargar los vectores, según lo dispuesto en [este](https://stackoverflow.com/questions/37793118/load-pretrained-glove-vectors-in-python) sitio.
- `nltk`: se usó para cargar el *opinion lexicon* descrito en el enunciado de manera conveniente. Se probó además sus métodos para realizar lematización, proceso que consiste en, dada una palabra flexionada (en forma plural, verbo conjugado, etc), encontrar su lema, es decir, algo similar a la forma en que sería encontrada en un diccionario.

In [ ]:
import pandas as pd
import csv
from nltk.stem import WordNetLemmatizer
import numpy as np
import nltk
nltk.download("stopwords")
nltk.download("opinion_lexicon")
nltk.download('punkt')
from nltk.corpus import opinion_lexicon
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [ ]:
GLOVE_PATH = "./glove.42B.300d.txt"

In [ ]:
NON_WORDS = list(punctuation)
NON_WORDS.extend(map(str, range(10)))

In [ ]:
ENGLISH_STOPWORDS = stopwords.words("english")

La forma de cargar el opinion lexicon es la descrita en la [documentación](http://www.nltk.org/api/nltk.corpus.reader.html?highlight=wordnet) de NLTK.

In [ ]:
positive_data = opinion_lexicon.positive()
negative_data = opinion_lexicon.negative()

Con el fin de tener una representación más general de las palabras, se convierte cada vocablo en las listas anteriores a minúsculas.

In [ ]:
positive_data = [word.lower() for word in positive_data]
negative_data = [word.lower() for word in negative_data]

A continuación, se carga la matriz de *embeddings*:

In [ ]:
glove_matrix = pd.read_csv(GLOVE_PATH, sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE, na_values=None, keep_default_na=False)

En el mismo recurso para cargar la matriz en memoria (adjunto en la descripción de las librerías utilizadas), se menciona una función que, dada una palabra, retorna su *embedding*. Esta función es `vec(w)`, definida a continuación:

In [ ]:
def vec(w): 
    return glove_matrix.loc[w].as_matrix()

## Breve análisis exploratorio

No se puede empezar a trabajar sin tener al menos una noción básica de la composición de la matriz y de las palabras con etiqueta. Por esta razón, en primer lugar cabe preguntarse cuántas palabras hay en la matriz de *embeddings*.

In [ ]:
glove_words = set(glove_matrix.index.tolist())
print(len(glove_words))

Podría ocurrir que no todas las palabras que tenemos etiquetadas según su polaridad tengan su representación vectorial en GloVe, por lo que conviene realizar el siguiente análisis:
Dada la unión entre las palabras de polaridad positiva y las de polaridad negativa, se revisa si están o no en la matriz. De no estar, se agregan a la lista `missing_words`, para decidir qué hacer con ellas futuramente.

In [ ]:
missing_words = set()
for word in set(positive_data).union(set(negative_data)):
    if word not in glove_words:
        missing_words.add(word)
print(len(missing_words))

Es posible notar que hay 161 palabras del *opinion lexicon* para las que no se tiene una representación vectorial. Antes de decidir qué hacer con estas palabras faltantes, conviene preguntarse si existen palabras que estén etiquetadas tanto de polaridad positiva como de polaridad negativa.

In [ ]:
wrong_tag = set(positive_data).intersection(negative_data)
print(wrong_tag)

Así, se evidencia que tres palabras relacionadas a la envidia parecen estar mal etiquetadas, pues no deberían aparecer entre las palabras positivas.

A continuación, se procede a realizar lematización de los vocablos positivos y negativos, para ver si existe algún cambio en la cantidad palabras sin *embedding* tras este cambio.

In [ ]:
positive_data_lemmatized = {nltk.stem.WordNetLemmatizer().lemmatize(word) for word in positive_data}
negative_data_lemmatized = {nltk.stem.WordNetLemmatizer().lemmatize(word) for word in negative_data}

In [ ]:
missing_words_lemmatized = set()
for word in positive_data_lemmatized.union(set(negative_data_lemmatized)):
    if word not in glove_words:
        missing_words_lemmatized.add(word)
print(len(missing_words_lemmatized))

In [ ]:
wrong_tag_lemmatized = set(positive_data_lemmatized).intersection(negative_data_lemmatized)
print(wrong_tag_lemmatized)

In [ ]:
positive_data_lemmatized = positive_data_lemmatized - wrong_tag_lemmatized

Al lematizar, es posible notar que hay una palabra menos que no tiene representación vectorial. En la siguiente celda, podrá evidenciarse qué palabra es la que hace la diferencia:

In [ ]:
print(missing_words - missing_words_lemmatized)

## Preprocesamiento de los datos

https://github.com/stanfordnlp/GloVe/search?utf8=%E2%9C%93&q=unk&type=

In [ ]:
positive_matrix = np.empty((0, 300))
for word in positive_data_lemmatized:
    if word in glove_words:
        positive_matrix = np.append(positive_matrix, [vec(word)], axis=0)
    else:
        positive_matrix = np.append(positive_matrix, [vec("unk")], axis=0)

In [ ]:
negative_matrix = np.empty((0, 300))
for word in negative_data_lemmatized:
    if word in glove_words:
        negative_matrix = np.append(negative_matrix, [vec(word)], axis=0)
    else:
        negative_matrix = np.append(negative_matrix, [vec("unk")], axis=0)

In [ ]:
print(positive_matrix.shape)
print(negative_matrix.shape)

In [ ]:
positive_labels = [1] * positive_matrix.shape[0]
negative_labels = [0] * negative_matrix.shape[0]
all_labels = positive_labels + negative_labels

In [ ]:
concatenated_matrix = np.concatenate([positive_matrix, negative_matrix], axis=0)
concatenated_matrix.shape

## Clasificación

### Separación entre entrenamiento y *testing*

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(concatenated_matrix, all_labels, test_size=0.2, random_state=0)

In [ ]:
def estimator_grid_search(estimator, parameters):
    clf = GridSearchCV(estimator=estimator, param_grid=parameters, n_jobs=-1, cv=10)
    clf.fit(X_train, y_train)   
    print("Mejor accuracy: {}".format(clf.best_score_))
    return clf

In [ ]:
def get_metrics(clf, X_test, y_test):
    metrics = {}
    predictions = clf.predict(X_test)
    metrics["accuracy"] = accuracy_score(y_test, predictions)
    metrics["precision"] = precision_score(y_test, predictions)
    metrics["recall"] = recall_score(y_test, predictions)
    metrics["f1-score"] = f1_score(y_test, predictions)
    return metrics

In [ ]:
def save_model(clf, clf_name):
    joblib.dump(clf, "best_{}.pkl".format(clf_name))

### SVM

In [ ]:
svm_params = {
    "C": [1, 10],
    "gamma": [0.001, 0.0001],
    "kernel": ["linear", "poly", "rbf", "sigmoid"]
}
tuning_svm = estimator_grid_search(SVC(), svm_params)
print("Mejor C: {}".format(tuning_svm.best_estimator_.C))
print("Mejor kernel: {}".format(tuning_svm.best_estimator_.kernel))
print("Mejor Gamma: {}".format(tuning_svm.best_estimator_.gamma))
final_svm = SVC(C=tuning_svm.best_estimator_.C, 
                kernel=tuning_svm.best_estimator_.kernel, 
                gamma=tuning_svm.best_estimator_.gamma, probability=True)
final_svm.fit(X_train, y_train)
svm_final_metrics = get_metrics(final_svm, X_test, y_test)
print(svm_final_metrics)
save_model(final_svm, "SVM")

### KNN

In [ ]:
knn_params = {"n_neighbors": np.arange(5) + 1, "algorithm": ["kd_tree", "ball_tree"]}
tuning_knn = estimator_grid_search(KNeighborsClassifier(), knn_params)
print("Mejor K: {}".format(tuning_knn.best_estimator_.n_neighbors)) 
print("Mejor algoritmo de búsqueda de vecinos: {}".format(tuning_knn.best_estimator_.algorithm))
final_knn = KNeighborsClassifier(n_neighbors=tuning_knn.best_estimator_.n_neighbors, 
                                 algorithm=tuning_knn.best_estimator_.algorithm)
final_knn.fit(X_train, y_train)
knn_final_metrics = get_metrics(final_knn, X_test, y_test)
print(knn_final_metrics)
save_model(final_knn, "KNN")

### Random Forest

In [ ]:
rf_params = {
    "n_estimators": np.arange(10) + 1, 
    "criterion": ["gini", "entropy"], 
    "max_features": ["sqrt", "log2", None]}
tuning_rf = estimator_grid_search(RandomForestClassifier(), rf_params)
print("Mejor cantidad de árboles: {}".format(tuning_rf.best_estimator_.n_estimators))
print("Mejor criterio de división: {}".format(tuning_rf.best_estimator_.criterion))
print("Mejor cantidad máxima de features: {}".format(tuning_rf.best_estimator_.max_features))
final_rf = RandomForestClassifier(n_estimators=tuning_rf.best_estimator_.n_estimators,
                                 criterion=tuning_rf.best_estimator_.criterion,
                                 max_features=tuning_rf.best_estimator_.max_features)
final_rf.fit(X_train, y_train)
rf_final_metrics = get_metrics(final_rf, X_test, y_test)
print(rf_final_metrics)
save_model(final_rf, "RF")

In [ ]:
def model_predict(clf, test_set):
    return clf.predict_proba(test_set)

## Pequeña prueba de modelos

In [ ]:
test_words = ["italian", "mexican", "black"]
test_words_matrix = np.empty((0, 300))
for word in test_words:
    test_words_matrix = np.append(test_words_matrix, [vec(word)], axis=0)
print("SVM: {}".format(model_predict(final_svm, test_words_matrix)))
print("KNN: {}".format(model_predict(final_knn, test_words_matrix)))
print("Random Forest: {}".format(model_predict(final_rf, test_words_matrix)))

In [ ]:
print(final_knn.predict(test_words_matrix))

## Predicción de oraciones

### Preprocesamiento de las oraciones

In [ ]:
def tokenize_lemmatize(sentence):
    sentence = sentence.lower()
    words_sentence = ''.join([c for c in sentence if c not in NON_WORDS])
    tokenized_sentence = word_tokenize(words_sentence)
    removed_stopwords = [word for word in tokenized_sentence if word not in ENGLISH_STOPWORDS]
    lemmatized_sentence = [nltk.stem.WordNetLemmatizer().lemmatize(word) for word in removed_stopwords]
    return tokenized_sentence

In [ ]:
def vectorize_sentence(sentence):
    sentence_matrix = np.empty((0, 300))
    for word in sentence:
        if word in glove_words:
            sentence_matrix = np.append(sentence_matrix, [vec(word)], axis=0)
        else:
            sentence_matrix = np.append(sentence_matrix, [vec("unk")], axis=0)
    mean_sentence_vector = sentence_matrix.mean(0)
    if np.isnan(np.sum(mean_sentence_vector)):
        mean_sentence_vector = vec("unk")
    return mean_sentence_vector

In [ ]:
def prepare_sentence(sentence):
    tokenized = tokenize_lemmatize(sentence)
    vectorized = vectorize_sentence(tokenized)
    return vectorized

### Pequeña prueba de predicción de oraciones

In [ ]:
best_svm = joblib.load("best_SVM.pkl")
print(model_predict(best_svm, list(map(prepare_sentence, ["Let's go get mexican food", 
                                                    "Let's go get italian food", 
                                                    "You're a faggot", 
                                                    "You are amazing", 
                                                    "You're amazing"]))))

### Evaluación del modelo sobre frases reales
https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences

In [ ]:
amazon = pd.read_csv("amazon_cells_labelled.txt", sep="\t", names=["sentence", "polarity"])
imdb = pd.read_csv("imdb_labelled.txt", sep="\t", names=["sentence", "polarity"])
yelp = pd.read_csv("yelp_labelled.txt", sep="\t", names=["sentence", "polarity"])

In [ ]:
all_sentences = pd.concat([amazon, imdb, yelp], sort=False)
all_sentences.head()

In [ ]:
X_sentence_test = all_sentences["sentence"].tolist()
y_sentence_test = all_sentences["polarity"]

In [ ]:
vectorized_sentences = list(map(prepare_sentence, X_sentence_test))

In [ ]:
new_sentences_matrix = np.empty((0, 300))
for sentence in X_sentence_test:
    vectorized_sentence = prepare_sentence(sentence)
    new_sentences_matrix = np.append(new_sentences_matrix, [vec("unk")], axis=0)

In [ ]:
new_concatenated_matrix = np.concatenate([positive_matrix, negative_matrix, new_sentences_matrix], axis=0)
new_all_labels = positive_labels + negative_labels + y_sentence_test.tolist()

In [ ]:
new_X_train, new_X_test, new_y_train, new_y_test = train_test_split(new_concatenated_matrix, new_all_labels, test_size=0.2, random_state=0)

In [ ]:
new_svm = SVC(C=tuning_svm.best_estimator_.C, 
                kernel=tuning_svm.best_estimator_.kernel, 
                gamma=tuning_svm.best_estimator_.gamma, probability=True)

In [ ]:
new_svm.fit(new_X_train, new_y_train)

In [ ]:
new_svm_final_metrics = get_metrics(final_svm, new_X_test, new_y_test)
print(new_svm_final_metrics)
save_model(new_svm, "sentences_SVM")